In [1]:
#All these packages need to be installed from pip
#For ML
import sklearn
import sklearn.feature_extraction.text
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups, make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer  #Feature extraction
from sklearn.naive_bayes import MultinomialNB #Our learner.
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, SelectFromModel
import pandas as pd

import nltk #For tokenizing and normalizing
import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
from matplotlib.colors import ListedColormap
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

%matplotlib inline

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import pandas
import requests
import json
import math

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pandas.read_pickle('cmv_full_features.pkl')
df = df.sample(frac = .1)
#splitting data
data_train, data_test = train_test_split(df, test_size=0.3, random_state=123)
data_train['is_train'] = True
data_test['is_train'] = False

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
#turn the training dataset into a tf-idf matrix
TFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=100, min_df=2, ngram_range=(1, 10),stop_words='english', norm='l2')
TFVects = TFVectorizer.fit_transform(data_train['com_text'])
TFVects.shape #(3836, 32241)
#print(TFVects.shape

(2685, 21494)

In [4]:
#Perform SVD on this matrix to reduce dimensionality

SVD = TruncatedSVD(n_components=1000, random_state=123)
reduced_data = SVD.fit_transform(TFVects)
reduced_data.shape

(2685, 1000)

In [5]:
#combining tfidf features with liguistic features and clustering labels
tfdf = pd.DataFrame(reduced_data)
features_train = pd.concat([tfdf, data_train.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
features_train.set_index('index', inplace = True)

In [6]:
#turn the test dataset into a tf-idf
TFVects_test = TFVectorizer.transform(data_test['com_text'])
reduced_test = SVD.transform(TFVects_test)
tfdf_test = pd.DataFrame(reduced_test)
features_test = pd.concat([tfdf_test, data_test.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
features_test.set_index('index', inplace = True)

In [7]:
X = features_train
y = data_train['com_delta_received'].as_matrix()

# Idea 1: try classification for all posts

## Logistic regression

In [8]:
#combining tfidf features with liguistic features and clustering labels
# tfdf = pd.DataFrame(TFVects.toarray())
# features_train = pd.concat([tfdf, data_train.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
# features_train.set_index('index', inplace = True)


In [9]:
#use scree plot to determine the number of dimensions
# n = TFVects.shape[0]
# fig = plt.figure(figsize=(12,5))
# ax1 = fig.add_subplot(121)
# eigen_vals = np.arange(n) + 1
# ax1.plot(eigen_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
# ax1.set_title('Scree Plot')
# ax1.set_xlabel('Principal Component')
# ax1.set_ylabel('Proportion of Explained Variance')

# ax2 = fig.add_subplot(122)
# eigen_vals = np.arange(10) + 1
# ax2.plot(eigen_vals, pca.explained_variance_ratio_[:10], 'ro-', linewidth=2)
# ax2.set_title('Scree Plot (First 20 Principal Components)')
# ax2.set_xlabel('Principal Component')
# ax2.set_ylabel('Proportion of Explained Variance')
# plt.show()

In [10]:
#predicting with first ten
# X = reduced_data[:, :2]
# Y = np.array([int(label) for label in data_train['com_delta_received']]) #Transform our predictor variable. 
              
#fitting logistic regresion
# logistic = linear_model.LogisticRegression()
# logistic.fit(X, Y)
# print("This logistic model using top two components fits {} of our training set".format(logistic.score(X,Y)))


In [11]:
# #turn the test dataset into a tf-idf
# TFVects_test = TFVectorizer.transform(data_test['com_text'])
# tfdf_test = pd.DataFrame(TFVects_test.toarray())
# features_test = pd.concat([tfdf_test, data_test.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
# features_test.set_index('index', inplace = True)
# reduced_data_test = pca.transform(features_test)

In [12]:
# X_test = reduced_data_test[:, :2]
# Y_test = np.array([int(label) for label in data_test['com_delta_received']])
# print("This logistic model using top ten components fits {} of our testing set".format(logistic.score(X_test, Y_test)))

In [13]:
# #Evaluation stats
# print("Precision:", sklearn.metrics.precision_score(Y, logistic.predict(X), average = 'weighted')) #precision
# print("Recall:",sklearn.metrics.recall_score(Y, logistic.predict(X), average = 'weighted')) #recall
# print("F-measure:",sklearn.metrics.f1_score(Y, logistic.predict(X), average = 'weighted')) #F-1 measure

In [14]:
# #ROC curve
# x, y, _ = sklearn.metrics.roc_curve(Y, logistic.predict_proba(X)[:,1])
# roc_auc = sklearn.metrics.auc(x,y)

# plt.figure()
# plt.plot(x,y, color = 'darkorange', lw = 2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic curve')
# plt.legend(loc="lower right")
# plt.show()

In [15]:
#Problem: we lost track of the feature names after PCA

## Binary Naive Bayes

## Decision tree

In [16]:
# tree = DecisionTreeClassifier(max_depth=4,random_state=0).fit(reduced_data, data_train['com_delta_received'])

In [17]:
# labels = tree.predict(reduced_data_test)

In [18]:
# mat = confusion_matrix(data_test['com_delta_received'], labels)
# seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#                 xticklabels=['delta_not_received', 'delta_received'], yticklabels=['delta_not_received', 'delta_received'])
# plt.xlabel('true label')
# plt.ylabel('predicted label');

In [19]:
# print('Precision: {}'.format(sklearn.metrics.precision_score(data_test['com_delta_received'], labels, average = 'weighted'))) 
# print('Recall: {}'.format(sklearn.metrics.recall_score(data_test['com_delta_received'], labels, average = 'weighted'))) 
# print('F1 Score: {}'.format(sklearn.metrics.f1_score(data_test['com_delta_received'], labels, average = 'weighted'))) 

# labels = [1 if dr else 0 for dr in labels]
# probs = tree.predict_proba(reduced_data_test)
# print('AUC Score: {}'.format(sklearn.metrics.roc_auc_score(data_test['com_delta_received'], probs[:,1])))



## Neural Nets - multi-layer perceptron

In [20]:
# #initialize the model
# mlp_clf = MLPClassifier()

# fit the model
# mlp_clf.fit(X, y)

# #mlp_labels = [mlp_clf.predict(v)[0] for v in features_test]

In [21]:
# #Confusion matrix
# mat = confusion_matrix(data_test['com_delta_received'], mlp_labels)
# seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#                 xticklabels=['delta_not_received', 'delta_received'], yticklabels=['delta_not_received', 'delta_received'])
# plt.xlabel('true label')
# plt.ylabel('predicted label')

In [22]:
#data_test['com_delta_received']
#mlp_labels

In [23]:
# #ROC curve
# x, y, _ = sklearn.metrics.roc_curve(data_test['com_delta_received'], mlp_labels)
# roc_auc = sklearn.metrics.auc(x,y)

# plt.figure()
# plt.plot(x,y, color = 'darkorange', lw = 2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic curve')
# plt.legend(loc="lower right")
# plt.show()

# Idea 2: try classification sparately for each "cluster"

# Feature extraction

Idea: We realized that none of the implementations of the machine learning algorithms actually provides us with the list of features that are most reliable to classify comments. Thus, we are using feature extraction tools from sklearn.

Oh, we've observed that the features are very stable across algorithms, we could interpret these features by finding them in comments and see if these comments receive delta or not

## feature extraction using SelectKBest

In [24]:
def extract_KBest(func, k):
    selector = SelectKBest(func, k=k).fit(X,y)
    feature_indices = selector.get_support(indices=True)
    
    selected_features = features_train.columns[feature_indices].get_values()
    scores = selector.scores_[feature_indices]
    if selector.pvalues_ != None:
        pvalues = selector.pvalues_[feature_indices]
    else:
        pvalues = None
    return (selected_features, scores, pvalues)

In [25]:
#f_classif
selected_features_f, scores_f, pvalues_f = extract_KBest(sklearn.feature_selection.f_classif, 20)
selected_features_f

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


array([0, 12, 38, 48, 60, 63, 111, 154, 223, 237, 278, 280, 292, 602, 640,
       827, 846, 905, 940, 'com_upvotes'], dtype=object)

In [26]:
#tf_features_indices_f = selected_features_f[:-1].astype(int)
#all_features_f  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_f].tolist()
#all_features_f.append(selected_features_f[-1])
#all_features_f
d = {'selected_features_f': selected_features_f, 'scores_f': scores_f, 'pvalues_f': pvalues_f}
F_f_DF = pd.DataFrame(data = d).sort(columns='scores_f', axis=0, ascending=False).reset_index(drop = True)
F_f_DF

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,pvalues_f,scores_f,selected_features_f
0,6.024069e-68,321.267992,com_upvotes
1,1.273266e-10,41.677239,0
2,1.543156e-04,14.359763,48
3,2.074398e-04,13.800399,827
4,3.706782e-04,12.706847,940
5,5.432522e-04,11.989956,292
6,6.585610e-04,11.629997,12
7,8.760924e-04,11.097625,63
8,1.973848e-03,9.592590,640
9,1.989941e-03,9.577626,280


In [27]:
#chi2 - we can't use this algo because some values of upvotes are negative


In [28]:
#mutual_info_classif
selected_features_m, scores_m, pvalues_m = extract_KBest(mutual_info_classif, 20)
selected_features_m

array([0, 2, 90, 135, 159, 195, 385, 471, 485, 523, 617, 682, 788, 846,
       852, 864, 888, 925, 'com_upvotes', 'com_avg_pt_depth'], dtype=object)

In [29]:
pvalues_m

In [30]:
#tf_features_indices_m = selected_features_m[:-1].astype(int)
#all_features_m  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_m].tolist()
#all_features_m.extend(selected_features_m[-1:])
#all_features_m
d = {'selected_features_m': selected_features_m, 'scores_m': scores_m}
F_m_DF = pd.DataFrame(data = d).sort(columns='scores_m', axis=0, ascending=False).reset_index(drop = True)
F_m_DF

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,scores_m,selected_features_m
0,0.034991,com_upvotes
1,0.010139,2
2,0.009390,925
3,0.008461,617
4,0.008429,846
5,0.008083,471
6,0.008082,0
7,0.007924,788
8,0.007884,385
9,0.007700,com_avg_pt_depth


In [31]:
F_m_DF

,scores_m,selected_features_m
0,0.034991,com_upvotes
1,0.010139,2
2,0.009390,925
3,0.008461,617
4,0.008429,846
5,0.008083,471
6,0.008082,0
7,0.007924,788
8,0.007884,385
9,0.007700,com_avg_pt_depth


## Feature extraction using Tree-based feature selection

In [32]:
def extract_model(clf_fitted):
    feature_indices = SelectFromModel(clf_fitted, prefit=True).get_support(indices=True)
    try:
        score = clf_fitted.feature_importances_[feature_indices]
    except:
        score = None
    selected_features = features_train.columns[feature_indices].get_values()
    return (selected_features, score)

In [33]:
#Decision tree
tree_clf = DecisionTreeClassifier(max_depth=10,random_state=0).fit(X, y)

In [34]:
selected_features_t1, scores_t1 = extract_model(tree_clf)
selected_features_t1

array([0, 59, 60, 108, 111, 331, 350, 401, 480, 489, 492, 533, 562, 565,
       592, 648, 654, 662, 797, 871, 904, 943, 954, 'com_upvotes'], dtype=object)

In [35]:
selected_features_t1.shape

(24,)

In [36]:
#tf_features_indices_t1 = selected_features_t1[:-2].astype(int)
#all_features_t1  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_t1].tolist()
#all_features_t1.extend(selected_features_t1[-2:])
#all_features_t1

d = {'selected_features_t1': selected_features_t1, 'scores_t1': scores_t1}
F_t1_DF = pd.DataFrame(data = d).sort(columns='scores_t1', axis=0, ascending=False).reset_index(drop = True)
F_t1_DF.iloc[:20, :]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,scores_t1,selected_features_t1
0,0.240499,com_upvotes
1,0.075325,60
2,0.056875,111
3,0.055495,533
4,0.055494,904
5,0.053587,401
6,0.049793,654
7,0.048181,871
8,0.038662,480
9,0.037565,648


In [37]:
#Extra decision tree
#fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset 
#and use averaging to improve the predictive accuracy and control over-fitting
tree2_clf = ExtraTreesClassifier(n_estimators=250,random_state=0).fit(X, y)

In [38]:
selected_features_t2, scores_t2 = extract_model(tree2_clf)
#selected_features_t2[2700:]

In [39]:
#selected_features_t2[2600:]

In [40]:
#tf_features_indices_t2 = selected_features_t2[:-5].astype(int)
#all_features_t2  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_t2].tolist()
#all_features_t2.extend(selected_features_t2[-5:])
#all_features_t1

d = {'selected_features_t2': selected_features_t2, 'scores_t2': scores_t2}
F_t2_DF = pd.DataFrame(data = d).sort(columns='scores_t2', axis=0, ascending=False).reset_index(drop = True)
F_t2_DF.iloc[:20, :]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,scores_t2,selected_features_t2
0,0.027824,com_upvotes
1,0.003684,0
2,0.003365,8
3,0.002979,57
4,0.002920,48
5,0.002761,63
6,0.002693,7
7,0.002555,6
8,0.002401,199
9,0.002392,44


In [41]:
F_DF = pd.concat([F_f_DF, F_m_DF, F_t1_DF.iloc[:20, :], F_t2_DF.iloc[:20, :]], axis =1)
F_DF

,pvalues_f,scores_f,selected_features_f,scores_m,selected_features_m,scores_t1,selected_features_t1,scores_t2,selected_features_t2
0,6.024069e-68,321.267992,com_upvotes,0.034991,com_upvotes,0.240499,com_upvotes,0.027824,com_upvotes
1,1.273266e-10,41.677239,0,0.010139,2,0.075325,60,0.003684,0
2,1.543156e-04,14.359763,48,0.009390,925,0.056875,111,0.003365,8
3,2.074398e-04,13.800399,827,0.008461,617,0.055495,533,0.002979,57
4,3.706782e-04,12.706847,940,0.008429,846,0.055494,904,0.002920,48
5,5.432522e-04,11.989956,292,0.008083,471,0.053587,401,0.002761,63
6,6.585610e-04,11.629997,12,0.008082,0,0.049793,654,0.002693,7
7,8.760924e-04,11.097625,63,0.007924,788,0.048181,871,0.002555,6
8,1.973848e-03,9.592590,640,0.007884,385,0.038662,480,0.002401,199
9,1.989941e-03,9.577626,280,0.007700,com_avg_pt_depth,0.037565,648,0.002392,44


In [42]:
## Feature extraction using Neural nets - mlp

# log_clf = linear_model.LogisticRegression()
# log_clf.fit(X, y)
# selected_features_log, scores_log = extract_model(log_clf)
# selected_features_log.shape
# selected_features_log[4500:]
# tf_features_indices_log = selected_features_log[:-1].astype(int)
# all_features_log = np.array(TFVectorizer.get_feature_names())[tf_features_indices_log].tolist()
# all_features_log.extend(selected_features_log[-1:])
#all_features_t1

# d = {'all_features_log': all_features_log, 'scores_log': scores_log}
# F_log_DF = pd.DataFrame(data = d).sort(columns='scores_log', axis=0, ascending=False).reset_index(drop = True)
# F_log_DF.iloc[:20, :]
#log_decisionDF = pd.DataFrame(log_clf.decision_function(X))#.sort(columns = [0], ascending = False)
#log_clf.densify(）